In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from glob import glob

# --- CONFIGURATION --- #
DATA_FOLDER = r"C:\Users\bmoha\Desktop\DataCollection\2150\heat_logs\2025-6-5_13161"
OUTPUT_FOLDER = os.path.join(DATA_FOLDER, "output")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- DEBUG: List all files --- #
all_files = glob(os.path.join(DATA_FOLDER, "*.*"))
print(f"\n🔍 All files in folder: {len(all_files)} total")
for f in all_files[:5]:
    print(" -", os.path.basename(f))

# --- Load only .csv files larger than 5 KB --- #
raw_csv_files = glob(os.path.join(DATA_FOLDER, "*.csv"))
print(f"\n📄 .csv files found: {len(raw_csv_files)}")

csv_files = [f for f in raw_csv_files if os.path.getsize(f) >= 1 * 1024]
print(f"✅ CSV files > 1 KB: {len(csv_files)}")

if not csv_files:
    print("⚠️ No valid CSV files found. Check path, file extensions, or size filter.")
else:
    print("\n📂 Starting processing...")

# --- Loop through CSV files --- #
for file_path in csv_files:
    file_name = os.path.basename(file_path)
    print(f"\n🔧 Processing: {file_name}")

    try:
        df = pd.read_csv(file_path)

        # Required columns check
        required_cols = {
            'time', 'position_x', 'position_y', 'position_z',
            'rotation_yaw', 'rotation_pitch', 'rotation_roll', 'Aname'
        }

        if not required_cols.issubset(df.columns):
            print(f"❌ Skipped {file_name}: Missing required columns.")
            continue

        # Filter head data
        head_df = df[df['Aname'].str.endswith('Head', na=False)]
        chris_head = head_df[head_df['Aname'].str.contains('BP_Chris_28', na=False)]
        pawn_head = head_df[head_df['Aname'].str.contains('myMRPawn', na=False)]

        if chris_head.empty or pawn_head.empty:
            print(f"⚠️ Skipped {file_name}: No head data for one or both characters.")
            continue

        # Merge on time
        merged = pd.merge(chris_head, pawn_head, on='time', suffixes=('_chris', '_pawn'))

        if merged.empty:
            print(f"⚠️ Skipped {file_name}: No common timestamps.")
            continue

        # Compute Euclidean distance
        merged['head_distance'] = np.sqrt(
            (merged['position_x_chris'] - merged['position_x_pawn'])**2 +
            (merged['position_y_chris'] - merged['position_y_pawn'])**2 +
            (merged['position_z_chris'] - merged['position_z_pawn'])**2
        )

        # Plot
        plt.figure(figsize=(10, 4))
        plt.plot(merged['time'], merged['head_distance'], color='green')
        plt.title(f'Head Distance Over Time\n{file_name}', fontsize=10)
        plt.xlabel('Time')
        plt.ylabel('Distance (units)')
        plt.grid(True)
        plt.tight_layout()

        # Save plot
        output_path = os.path.join(OUTPUT_FOLDER, file_name.replace('.csv', '_head_distance.png'))
        plt.savefig(output_path)
        plt.close()

        print(f"✅ Plot saved to: {output_path}")

    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")



🔍 All files in folder: 0 total

📄 .csv files found: 0
✅ CSV files > 1 KB: 0
⚠️ No valid CSV files found. Check path, file extensions, or size filter.


In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === PATH CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Change to your root path
OUTPUT_DIR = os.path.join(ROOT, "sync_plots")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_pitch_sync(df1, df2, participant_id, session, body_part, char1_name, char2_name):
    # Merge on time
    merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
    if merged.empty:
        print(f"⚠️ No overlapping timestamps for {participant_id} — {session} — {body_part}")
        return

    pitch_chris = normalize(merged["rotation_pitch_chris"])
    pitch_pawn = normalize(merged["rotation_pitch_pawn"])
    time = merged["time"]

    # Synchrony Calculation
    pitch_diff = np.abs(pitch_chris - pitch_pawn)
    sync_threshold = 0.15
    in_sync = pitch_diff < sync_threshold
    sync_ratio = in_sync.sum() / len(in_sync) * 100  # in percent

    # Plot
    plt.figure(figsize=(12, 5))
    plt.plot(time, pitch_chris, label=f"{char1_name} Pitch", color="blue")
    plt.plot(time, pitch_pawn, label=f"{char2_name} Pitch", color="orange")
    plt.fill_between(time, pitch_chris, pitch_pawn, where=in_sync,
                     color="green", alpha=0.3,
                     label=f"In Sync (ΔPitch < 0.15) — {sync_ratio:.1f}%")

    plt.title(f"{body_part} Pitch Synchrony\nParticipant {participant_id} — Session {session}")
    plt.xlabel("Time")
    plt.ylabel("Normalized Pitch")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Save
    filename = f"{participant_id}_{session}_{body_part}_pitch_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, filename), dpi=300)
    plt.close()

# === MAIN ANALYSIS LOOP ===

for root, dirs, files in os.walk(ROOT):
    if os.path.basename(root) == "heat_logs":
        participant_id = root.split(os.sep)[-3]

        for session_folder in os.listdir(root):
            session_path = os.path.join(root, session_folder)
            if not os.path.isdir(session_path):
                continue

            # Load all .csv files in the session
            dfs = []
            for file in os.listdir(session_path):
                if file.endswith(".csv"):
                    try:
                        df = pd.read_csv(os.path.join(session_path, file))
                        dfs.append(df)
                    except Exception as e:
                        print(f"❌ Could not read {file} — {e}")

            if not dfs:
                continue

            session_df = pd.concat(dfs, ignore_index=True)

            # Check required columns
            required_cols = {"time", "rotation_pitch", "Aname"}
            if not required_cols.issubset(session_df.columns):
                print(f"⚠️ Missing required columns in {participant_id} — {session_folder}")
                continue

            # Extract character IDs and body parts
            session_df["char_id"] = session_df["Aname"].apply(extract_character_id)
            session_df["body_part"] = session_df["Aname"].apply(lambda x: x.split("_")[-1])
            characters = session_df["char_id"].unique()

            if len(characters) < 2:
                print(f"⚠️ Less than 2 characters found in {participant_id} — {session_folder}")
                continue

            char1, char2 = characters[:2]

            # Analyze all 3 body parts
            for body_part in ["Head", "WristLeft", "WristRight"]:
                df1 = session_df[(session_df["char_id"] == char1) & (session_df["body_part"] == body_part)]
                df2 = session_df[(session_df["char_id"] == char2) & (session_df["body_part"] == body_part)]

                # Select only time and pitch
                df1 = df1[["time", "rotation_pitch"]].drop_duplicates(subset="time").sort_values("time")
                df2 = df2[["time", "rotation_pitch"]].drop_duplicates(subset="time").sort_values("time")

                df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
                df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

                if not df1.empty and not df2.empty:
                    plot_pitch_sync(df1, df2, participant_id, session_folder, body_part, char1, char2)
                else:
                    print(f"⚠️ Missing pitch data for {participant_id} — {session_folder} — {body_part}")


⚠️ Missing pitch data for DataCollection — 2025-6-7_115325 — Head
⚠️ Missing pitch data for DataCollection — 2025-6-7_115325 — WristRight
⚠️ Less than 2 characters found in DataCollection — 2025-6-7_17131
⚠️ Less than 2 characters found in DataCollection — 2025-6-4_131612
⚠️ Less than 2 characters found in DataCollection — 2025-6-4_142625
⚠️ Less than 2 characters found in DataCollection — 2025-6-6_9201
⚠️ Less than 2 characters found in DataCollection — 2025-6-6_9957
⚠️ Less than 2 characters found in DataCollection — 2025-6-4_10021
⚠️ Less than 2 characters found in DataCollection — 2025-6-4_111057
⚠️ Missing required columns in DataCollection — 2025-6-4_111431
⚠️ Missing pitch data for DataCollection — 2025-6-5_16598 — Head
⚠️ Missing pitch data for DataCollection — 2025-6-5_16598 — WristRight


In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Update this path as needed
OUTPUT_DIR = os.path.join(ROOT, "sync_plots")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_pitch_sync(df1, df2, participant_id, session, body_part, char1_name, char2_name):
    merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
    if merged.empty:
        print(f"⚠️ No overlapping timestamps for {participant_id} — {session} — {body_part}")
        return

    pitch_chris = normalize(merged["rotation_pitch_chris"])
    pitch_pawn = normalize(merged["rotation_pitch_pawn"])
    time = merged["time"]

    pitch_diff = np.abs(pitch_chris - pitch_pawn)
    sync_threshold = 0.15
    in_sync = pitch_diff < sync_threshold
    sync_ratio = in_sync.sum() / len(in_sync) * 100  # percentage

    # Plot
    plt.figure(figsize=(12, 5))
    plt.plot(time, pitch_chris, label=f"{char1_name} Pitch", color="blue")
    plt.plot(time, pitch_pawn, label=f"{char2_name} Pitch", color="orange")
    plt.fill_between(time, pitch_chris, pitch_pawn, where=in_sync,
                     color="green", alpha=0.3,
                     label=f"In Sync (ΔPitch < 0.15) — {sync_ratio:.1f}%")

    plt.title(f"{body_part} Pitch Synchrony — Participant {participant_id}\nSession: {session}")
    plt.xlabel(f"Participant {participant_id}")
    plt.ylabel("Normalized Pitch")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    filename = f"{participant_id}_{session}_{body_part}_pitch_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, filename), dpi=300)
    plt.close()

# === MAIN SCRIPT ===

for root, dirs, files in os.walk(ROOT):
    if os.path.basename(root) == "heat_logs":
        participant_id = root.split(os.sep)[-3]

        for session_folder in os.listdir(root):
            session_path = os.path.join(root, session_folder)
            if not os.path.isdir(session_path):
                continue

            # Combine all CSVs for this session
            dfs = []
            for file in os.listdir(session_path):
                if file.endswith(".csv"):
                    try:
                        df = pd.read_csv(os.path.join(session_path, file))
                        dfs.append(df)
                    except Exception as e:
                        print(f"❌ Could not read {file} — {e}")

            if not dfs:
                continue

            session_df = pd.concat(dfs, ignore_index=True)

            # Ensure necessary columns exist
            if not {"time", "rotation_pitch", "Aname"}.issubset(session_df.columns):
                print(f"⚠️ Missing columns in session {participant_id} — {session_folder}")
                continue

            # Extract character and body part identifiers
            session_df["char_id"] = session_df["Aname"].apply(extract_character_id)
            session_df["body_part"] = session_df["Aname"].apply(lambda x: x.split("_")[-1])

            characters = session_df["char_id"].unique()
            if len(characters) < 2:
                print(f"⚠️ Less than 2 characters found in {participant_id} — {session_folder}")
                continue

            char1, char2 = characters[:2]  # Choose first 2 characters

            # Analyze each body part separately
            for body_part in ["Head", "WristLeft", "WristRight"]:
                df1 = session_df[(session_df["char_id"] == char1) & (session_df["body_part"] == body_part)]
                df2 = session_df[(session_df["char_id"] == char2) & (session_df["body_part"] == body_part)]

                df1 = df1[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")
                df2 = df2[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")

                df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
                df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

                if not df1.empty and not df2.empty:
                    plot_pitch_sync(df1, df2, participant_id, session_folder, body_part, char1, char2)
                else:
                    print(f"⚠️ Missing pitch data for {participant_id} — {session_folder} — {body_part}")


⚠️ Missing pitch data for DataCollection — 2025-6-7_115325 — Head
⚠️ Missing pitch data for DataCollection — 2025-6-7_115325 — WristRight
⚠️ Less than 2 characters found in DataCollection — 2025-6-7_17131
⚠️ Less than 2 characters found in DataCollection — 2025-6-4_131612
⚠️ Less than 2 characters found in DataCollection — 2025-6-4_142625
⚠️ Less than 2 characters found in DataCollection — 2025-6-6_9201
⚠️ Less than 2 characters found in DataCollection — 2025-6-6_9957
⚠️ Less than 2 characters found in DataCollection — 2025-6-4_10021
⚠️ Less than 2 characters found in DataCollection — 2025-6-4_111057
⚠️ Missing columns in session DataCollection — 2025-6-4_111431
⚠️ Missing pitch data for DataCollection — 2025-6-5_16598 — Head
⚠️ Missing pitch data for DataCollection — 2025-6-5_16598 — WristRight


In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Update this path as needed
OUTPUT_DIR = os.path.join(ROOT, "sync_plots")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_pitch_sync(df1, df2, participant_id, body_part, char1_name, char2_name):
    merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
    if merged.empty:
        print(f"⚠️ No overlapping timestamps for {participant_id} — {body_part}")
        return

    pitch_chris = normalize(merged["rotation_pitch_chris"])
    pitch_pawn = normalize(merged["rotation_pitch_pawn"])
    time = merged["time"]

    pitch_diff = np.abs(pitch_chris - pitch_pawn)
    sync_threshold = 0.15
    in_sync = pitch_diff < sync_threshold
    sync_ratio = in_sync.sum() / len(in_sync) * 100  # percentage

    # Plot
    plt.figure(figsize=(12, 5))
    plt.plot(time, pitch_chris, label=f"{char1_name} Pitch", color="blue")
    plt.plot(time, pitch_pawn, label=f"{char2_name} Pitch", color="orange")
    plt.fill_between(time, pitch_chris, pitch_pawn, where=in_sync,
                     color="green", alpha=0.3,
                     label=f"In Sync (ΔPitch < 0.15) — {sync_ratio:.1f}%")

    plt.title(f"{body_part} Pitch Synchrony — Participant {participant_id}")
    plt.xlabel(f"Participant {participant_id}")
    plt.ylabel("Normalized Pitch")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Save as: 2150_Head_sync.png
    filename = f"{participant_id}_{body_part}_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, filename), dpi=300)
    plt.close()

# === MAIN SCRIPT ===

# Group data per participant and body part across sessions
participant_data = defaultdict(list)

for root, dirs, files in os.walk(ROOT):
    if os.path.basename(root) == "heat_logs":
        participant_id = root.split(os.sep)[-3]

        for session_folder in os.listdir(root):
            session_path = os.path.join(root, session_folder)
            if not os.path.isdir(session_path):
                continue

            # Load all CSVs in this session
            dfs = []
            for file in os.listdir(session_path):
                if file.endswith(".csv"):
                    try:
                        df = pd.read_csv(os.path.join(session_path, file))
                        dfs.append(df)
                    except Exception as e:
                        print(f"❌ Could not read {file} — {e}")

            if not dfs:
                continue

            session_df = pd.concat(dfs, ignore_index=True)

            if not {"time", "rotation_pitch", "Aname"}.issubset(session_df.columns):
                print(f"⚠️ Skipping {participant_id} — {session_folder}: missing required columns")
                continue

            session_df["char_id"] = session_df["Aname"].apply(extract_character_id)
            session_df["body_part"] = session_df["Aname"].apply(lambda x: x.split("_")[-1])

            participant_data[participant_id].append(session_df)

# Now merge and plot per participant and body part
for participant_id, all_dfs in participant_data.items():
    full_df = pd.concat(all_dfs, ignore_index=True)
    characters = full_df["char_id"].unique()

    if len(characters) < 2:
        print(f"⚠️ Less than 2 characters found in {participant_id}, skipping.")
        continue

    char1, char2 = characters[:2]

    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
        df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

        if not df1.empty and not df2.empty:
            plot_pitch_sync(df1, df2, participant_id, body_part, char1, char2)
        else:
            print(f"⚠️ Missing pitch data for {participant_id} — {body_part}")


⚠️ Skipping DataCollection — 2025-6-4_111431: missing required columns


In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Adjust this path
OUTPUT_DIR = os.path.join(ROOT, "sync_plots")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_pitch_sync(df1, df2, participant_id, body_part, char1_name, char2_name):
    merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
    if merged.empty:
        print(f"⚠️ No overlapping timestamps for {participant_id} — {body_part}")
        return

    pitch_chris = normalize(merged["rotation_pitch_chris"])
    pitch_pawn = normalize(merged["rotation_pitch_pawn"])
    time = merged["time"]

    pitch_diff = np.abs(pitch_chris - pitch_pawn)
    sync_threshold = 0.15
    in_sync = pitch_diff < sync_threshold
    sync_ratio = in_sync.sum() / len(in_sync) * 100  # percentage

    # Plot
    plt.figure(figsize=(12, 5))
    plt.plot(time, pitch_chris, label=f"{char1_name} Pitch", color="blue")
    plt.plot(time, pitch_pawn, label=f"{char2_name} Pitch", color="orange")
    plt.fill_between(time, pitch_chris, pitch_pawn, where=in_sync,
                     color="green", alpha=0.3,
                     label=f"In Sync (ΔPitch < 0.15) — {sync_ratio:.1f}%")

    plt.title(f"{body_part} Pitch Synchrony — Participant {participant_id}")
    plt.xlabel(f"Participant {participant_id}")
    plt.ylabel("Normalized Pitch")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    filename = f"{participant_id}_{body_part}_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, filename), dpi=300)
    plt.close()

# === MAIN SCRIPT ===

# Step 1: Collect all heat_log folders and organize by participant
participant_data = defaultdict(list)
total_participants = set()

print("🔍 Scanning folder structure...")

for dirpath, dirnames, filenames in os.walk(ROOT):
    if os.path.basename(dirpath) == "heat_logs":
        try:
            participant_id = dirpath.split(os.sep)[-3]
            total_participants.add(participant_id)
        except IndexError:
            print(f"⚠️ Skipping path {dirpath} — couldn't extract participant ID")
            continue

        for session_folder in os.listdir(dirpath):
            session_path = os.path.join(dirpath, session_folder)
            if not os.path.isdir(session_path):
                continue

            csv_files = [f for f in os.listdir(session_path) if f.endswith(".csv")]
            if not csv_files:
                print(f"⚠️ No CSVs in {participant_id}/{session_folder}")
                continue

            session_dfs = []
            for file in csv_files:
                try:
                    df = pd.read_csv(os.path.join(session_path, file))
                    session_dfs.append(df)
                except Exception as e:
                    print(f"❌ Failed to read {file} in {participant_id}/{session_folder}: {e}")

            if session_dfs:
                session_df = pd.concat(session_dfs, ignore_index=True)
                participant_data[participant_id].append(session_df)

# Step 2: Process each participant
print(f"✅ Found {len(participant_data)} participants with data.\n")

for participant_id, dfs in participant_data.items():
    print(f"📊 Processing Participant {participant_id} ...")

    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_pitch", "Aname"}.issubset(full_df.columns):
        print(f"⚠️ Skipping {participant_id}: missing required columns.")
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()

    if len(characters) < 2:
        print(f"⚠️ Skipping {participant_id}: less than two characters found.")
        continue

    char1, char2 = characters[:2]

    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
        df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

        if not df1.empty and not df2.empty:
            plot_pitch_sync(df1, df2, participant_id, body_part, char1, char2)
        else:
            print(f"⚠️ No valid pitch data for {participant_id} — {body_part}")

print("\n✅ Done! All plots saved to:", OUTPUT_DIR)


🔍 Scanning folder structure...
⚠️ No CSVs in DataCollection/2025-6-5_101933
⚠️ No CSVs in DataCollection/2025-6-5_102122
⚠️ No CSVs in DataCollection/2025-6-5_13161
⚠️ No CSVs in DataCollection/2025-6-4_83728
⚠️ No CSVs in 3981/2025-6-5_152940
⚠️ No CSVs in 3981/2025-6-5_153025
⚠️ No CSVs in DataCollection/2025-6-5_152940
⚠️ No CSVs in DataCollection/2025-6-5_153025
⚠️ No CSVs in DataCollection/2025-6-5_124511
⚠️ No CSVs in DataCollection/2025-6-5_124612
⚠️ No CSVs in DataCollection/2025-6-5_124740
⚠️ No CSVs in DataCollection/2025-6-5_12527
⚠️ No CSVs in DataCollection/2025-6-6_142335
⚠️ No CSVs in DataCollection/2025-6-7_13249
⚠️ No CSVs in DataCollection/2025-6-6_154812
✅ Found 2 participants with data.

📊 Processing Participant DataCollection ...
📊 Processing Participant 3981 ...

✅ Done! All plots saved to: C:\Users\bmoha\Desktop\DataCollection\sync_plots


In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Update this if needed
OUTPUT_DIR = os.path.join(ROOT, "sync_plots")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_pitch_sync(df1, df2, participant_id, body_part, char1_name, char2_name):
    merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
    if merged.empty:
        print(f"⚠️ No overlapping timestamps for {participant_id} — {body_part}")
        return

    pitch_chris = normalize(merged["rotation_pitch_chris"])
    pitch_pawn = normalize(merged["rotation_pitch_pawn"])
    time = merged["time"]

    pitch_diff = np.abs(pitch_chris - pitch_pawn)
    sync_threshold = 0.15
    in_sync = pitch_diff < sync_threshold
    sync_ratio = in_sync.sum() / len(in_sync) * 100

    plt.figure(figsize=(12, 5))
    plt.plot(time, pitch_chris, label=f"{char1_name} Pitch", color="blue")
    plt.plot(time, pitch_pawn, label=f"{char2_name} Pitch", color="orange")
    plt.fill_between(time, pitch_chris, pitch_pawn, where=in_sync,
                     color="green", alpha=0.3,
                     label=f"In Sync (ΔPitch < 0.15) — {sync_ratio:.1f}%")

    plt.title(f"{body_part} Pitch Synchrony — Participant {participant_id}")
    plt.xlabel(f"Participant {participant_id}")
    plt.ylabel("Normalized Pitch")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    filename = f"{participant_id}_{body_part}_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, filename), dpi=300)
    plt.close()

# === MAIN SCRIPT ===

participant_data = defaultdict(list)

print("🔍 Scanning folder structure...")

for participant_id in os.listdir(ROOT):
    participant_path = os.path.join(ROOT, participant_id)
    if not os.path.isdir(participant_path):
        continue

    heat_logs_path = os.path.join(participant_path, "heat_logs")
    if not os.path.exists(heat_logs_path):
        continue

    for session_name in os.listdir(heat_logs_path):
        session_path = os.path.join(heat_logs_path, session_name)
        if not os.path.isdir(session_path):
            continue

        excel_files = [f for f in os.listdir(session_path) if f.endswith(".xlsx")]
        if not excel_files:
            print(f"⚠️ No Excel files in {participant_id}/{session_name}")
            continue

        session_dfs = []
        for file in excel_files:
            file_path = os.path.join(session_path, file)
            try:
                df = pd.read_excel(file_path)
                session_dfs.append(df)
            except Exception as e:
                print(f"❌ Failed to read {file_path}: {e}")

        if session_dfs:
            session_df = pd.concat(session_dfs, ignore_index=True)
            participant_data[participant_id].append(session_df)

# === PROCESS PARTICIPANTS ===

print(f"\n✅ Found {len(participant_data)} participants with Excel data.\n")

for participant_id, dfs in participant_data.items():
    print(f"📊 Processing Participant {participant_id} ...")

    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_pitch", "Aname"}.issubset(full_df.columns):
        print(f"⚠️ Skipping {participant_id}: missing required columns.")
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()

    if len(characters) < 2:
        print(f"⚠️ Skipping {participant_id}: less than two characters found.")
        continue

    char1, char2 = characters[:2]

    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
        df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

        if not df1.empty and not df2.empty:
            plot_pitch_sync(df1, df2, participant_id, body_part, char1, char2)
        else:
            print(f"⚠️ No pitch data for {participant_id} — {body_part}")

print("\n✅ Done! All plots saved to:", OUTPUT_DIR)


🔍 Scanning folder structure...
⚠️ No Excel files in 1060/2025-6-5_13161
⚠️ No Excel files in 2150/2025-6-5_101933
⚠️ No Excel files in 2150/2025-6-5_102122
⚠️ No Excel files in 2150/2025-6-5_102239
⚠️ No Excel files in 2150/2025-6-5_103840
⚠️ No Excel files in 2150/2025-6-5_13161
⚠️ No Excel files in 2182/2025-6-6_111420
⚠️ No Excel files in 2226/2025-6-7_111516
⚠️ No Excel files in 2226/2025-6-7_115325
⚠️ No Excel files in 2462/2025-6-4_174114
⚠️ No Excel files in 2973/2025-6-7_17131
⚠️ No Excel files in 3154/2025-6-4_131612
⚠️ No Excel files in 3154/2025-6-4_142625
⚠️ No Excel files in 3291/2025-6-6_9201
⚠️ No Excel files in 3291/2025-6-6_9957
⚠️ No Excel files in 3709/2025-6-4_10021
⚠️ No Excel files in 3709/2025-6-4_83728
⚠️ No Excel files in 3814/2025-6-4_111057
⚠️ No Excel files in 3814/2025-6-4_111431
⚠️ No Excel files in 4050/2025-6-5_152940
⚠️ No Excel files in 4050/2025-6-5_153025
⚠️ No Excel files in 4050/2025-6-5_153216
⚠️ No Excel files in 4050/2025-6-5_172837
⚠️ No Excel 

In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # <- Adjust this if needed
OUTPUT_DIR = os.path.join(ROOT, "sync_plots")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_pitch_sync(df1, df2, participant_id, body_part, char1_name, char2_name):
    merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
    if merged.empty:
        print(f"⚠️ No overlapping timestamps for {participant_id} — {body_part}")
        return

    pitch_chris = normalize(merged["rotation_pitch_chris"])
    pitch_pawn = normalize(merged["rotation_pitch_pawn"])
    time = merged["time"]

    pitch_diff = np.abs(pitch_chris - pitch_pawn)
    sync_threshold = 0.15
    in_sync = pitch_diff < sync_threshold
    sync_ratio = in_sync.sum() / len(in_sync) * 100

    plt.figure(figsize=(12, 5))
    plt.plot(time, pitch_chris, label=f"{char1_name} Pitch", color="blue")
    plt.plot(time, pitch_pawn, label=f"{char2_name} Pitch", color="orange")
    plt.fill_between(time, pitch_chris, pitch_pawn, where=in_sync,
                     color="green", alpha=0.3,
                     label=f"In Sync (ΔPitch < 0.15) — {sync_ratio:.1f}%")

    plt.title(f"{body_part} Pitch Synchrony — Participant {participant_id}")
    plt.xlabel(f"Participant {participant_id}")
    plt.ylabel("Normalized Pitch")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    filename = f"{participant_id}_{body_part}_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, filename), dpi=300)
    plt.close()

# === MAIN SCRIPT ===

participant_data = defaultdict(list)

print("🔍 Scanning folder structure...")

for participant_id in os.listdir(ROOT):
    participant_path = os.path.join(ROOT, participant_id)
    if not os.path.isdir(participant_path):
        continue

    heat_logs_path = os.path.join(participant_path, "heat_logs")
    if not os.path.exists(heat_logs_path):
        continue

    for session_name in os.listdir(heat_logs_path):
        session_path = os.path.join(heat_logs_path, session_name)
        if not os.path.isdir(session_path):
            continue

        csv_files = [f for f in os.listdir(session_path) if f.endswith(".csv")]
        if not csv_files:
            print(f"⚠️ No CSV files in {participant_id}/{session_name}")
            continue

        session_dfs = []
        for file in csv_files:
            file_path = os.path.join(session_path, file)
            try:
                df = pd.read_csv(file_path)
                session_dfs.append(df)
            except Exception as e:
                print(f"❌ Failed to read {file_path}: {e}")

        if session_dfs:
            session_df = pd.concat(session_dfs, ignore_index=True)
            participant_data[participant_id].append(session_df)

# === PROCESS PARTICIPANTS ===

print(f"\n✅ Found {len(participant_data)} participants with CSV data.\n")

for participant_id, dfs in participant_data.items():
    print(f"📊 Processing Participant {participant_id} ...")

    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_pitch", "Aname"}.issubset(full_df.columns):
        print(f"⚠️ Skipping {participant_id}: missing required columns.")
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()

    if len(characters) < 2:
        print(f"⚠️ Skipping {participant_id}: less than two characters found.")
        continue

    char1, char2 = characters[:2]

    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
        df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

        if not df1.empty and not df2.empty:
            plot_pitch_sync(df1, df2, participant_id, body_part, char1, char2)
        else:
            print(f"⚠️ No pitch data for {participant_id} — {body_part}")

print("\n✅ Done! All plots saved to:", OUTPUT_DIR)


🔍 Scanning folder structure...
⚠️ No CSV files in 2150/2025-6-5_101933
⚠️ No CSV files in 2150/2025-6-5_102122
⚠️ No CSV files in 2150/2025-6-5_13161
⚠️ No CSV files in 3709/2025-6-4_83728
⚠️ No CSV files in 4050/2025-6-5_152940
⚠️ No CSV files in 4050/2025-6-5_153025
⚠️ No CSV files in 5390/2025-6-5_124511
⚠️ No CSV files in 5390/2025-6-5_124612
⚠️ No CSV files in 5390/2025-6-5_124740
⚠️ No CSV files in 5390/2025-6-5_12527
⚠️ No CSV files in 5985/2025-6-6_142335
⚠️ No CSV files in 9084/2025-6-7_13249
⚠️ No CSV files in 9383/2025-6-6_154812

✅ Found 24 participants with CSV data.

📊 Processing Participant 1060 ...
📊 Processing Participant 2150 ...
📊 Processing Participant 2182 ...
📊 Processing Participant 2226 ...
📊 Processing Participant 2462 ...
📊 Processing Participant 2973 ...
⚠️ Skipping 2973: less than two characters found.
📊 Processing Participant 3154 ...
⚠️ Skipping 3154: less than two characters found.
📊 Processing Participant 3291 ...
⚠️ Skipping 3291: less than two characte

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"
OUTPUT_DIR = os.path.join(ROOT, "sync_plots")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_combined_sync(df_dict, participant_id, char1_name, char2_name):
    body_parts = ["Head", "WristLeft", "WristRight"]

    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    fig.suptitle(f"Pitch Synchrony — Participant {participant_id}", fontsize=16)

    for idx, body_part in enumerate(body_parts):
        df1, df2 = df_dict.get(body_part, (None, None))
        ax = axes[idx]

        if df1 is None or df2 is None or df1.empty or df2.empty:
            ax.set_title(f"{body_part}: No Data")
            ax.axis("off")
            continue

        merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
        if merged.empty:
            ax.set_title(f"{body_part}: No Overlapping Time")
            ax.axis("off")
            continue

        pitch_chris = normalize(merged["rotation_pitch_chris"])
        pitch_pawn = normalize(merged["rotation_pitch_pawn"])
        time = merged["time"]

        pitch_diff = np.abs(pitch_chris - pitch_pawn)
        in_sync = pitch_diff < 0.15
        sync_ratio = in_sync.sum() / len(in_sync) * 100

        ax.plot(time, pitch_chris, label=f"{char1_name} Pitch", color="blue")
        ax.plot(time, pitch_pawn, label=f"{char2_name} Pitch", color="orange")
        ax.fill_between(time, pitch_chris, pitch_pawn, where=in_sync,
                        color="green", alpha=0.3,
                        label=f"In Sync — {sync_ratio:.1f}%")
        ax.set_title(f"{body_part} (Sync: {sync_ratio:.1f}%)")
        ax.set_ylabel("Normalized Pitch")
        ax.grid(True)
        ax.legend()

    plt.xlabel(f"Participant {participant_id}")
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    fname = f"{participant_id}_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, fname), dpi=300)
    plt.close()

# === MAIN SCRIPT ===

participant_data = defaultdict(list)

print("🔍 Scanning folder structure...")

for participant_id in os.listdir(ROOT):
    participant_path = os.path.join(ROOT, participant_id)
    if not os.path.isdir(participant_path):
        continue

    heat_logs_path = os.path.join(participant_path, "heat_logs")
    if not os.path.exists(heat_logs_path):
        continue

    for session_name in os.listdir(heat_logs_path):
        session_path = os.path.join(heat_logs_path, session_name)
        if not os.path.isdir(session_path):
            continue

        csv_files = [f for f in os.listdir(session_path) if f.endswith(".csv")]
        if not csv_files:
            print(f"⚠️ No CSV files in {participant_id}/{session_name}")
            continue

        session_dfs = []
        for file in csv_files:
            file_path = os.path.join(session_path, file)
            try:
                df = pd.read_csv(file_path)
                session_dfs.append(df)
            except Exception as e:
                print(f"❌ Failed to read {file_path}: {e}")

        if session_dfs:
            session_df = pd.concat(session_dfs, ignore_index=True)
            participant_data[participant_id].append(session_df)

print(f"\n✅ Found {len(participant_data)} participants with CSV data.\n")

for participant_id, dfs in participant_data.items():
    print(f"📊 Processing Participant {participant_id} ...")

    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_pitch", "Aname"}.issubset(full_df.columns):
        print(f"⚠️ Skipping {participant_id}: missing required columns.")
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()

    if len(characters) < 2:
        print(f"⚠️ Skipping {participant_id}: less than two characters found.")
        continue

    char1, char2 = characters[:2]

    df_dict = {}
    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
        df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

        df_dict[body_part] = (df1, df2)

    plot_combined_sync(df_dict, participant_id, char1, char2)

print("\n✅ All plots saved to:", OUTPUT_DIR)


🔍 Scanning folder structure...
⚠️ No CSV files in 2150/2025-6-5_101933
⚠️ No CSV files in 2150/2025-6-5_102122
⚠️ No CSV files in 2150/2025-6-5_13161
⚠️ No CSV files in 3709/2025-6-4_83728
⚠️ No CSV files in 4050/2025-6-5_152940
⚠️ No CSV files in 4050/2025-6-5_153025
⚠️ No CSV files in 5390/2025-6-5_124511
⚠️ No CSV files in 5390/2025-6-5_124612
⚠️ No CSV files in 5390/2025-6-5_124740
⚠️ No CSV files in 5390/2025-6-5_12527
⚠️ No CSV files in 5985/2025-6-6_142335
⚠️ No CSV files in 9084/2025-6-7_13249
⚠️ No CSV files in 9383/2025-6-6_154812

✅ Found 24 participants with CSV data.

📊 Processing Participant 1060 ...
📊 Processing Participant 2150 ...
📊 Processing Participant 2182 ...
📊 Processing Participant 2226 ...
📊 Processing Participant 2462 ...
📊 Processing Participant 2973 ...
⚠️ Skipping 2973: less than two characters found.
📊 Processing Participant 3154 ...
⚠️ Skipping 3154: less than two characters found.
📊 Processing Participant 3291 ...
⚠️ Skipping 3291: less than two characte

In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Change if needed
OUTPUT_DIR = os.path.join(ROOT, "sync_plots")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_combined_sync(df_dict, participant_id, char1_name, char2_name):
    body_parts = {
        "Head": "🧠 Head",
        "WristLeft": "🤚 WristLeft",
        "WristRight": "✋ WristRight"
    }

    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    fig.suptitle(f"🎚 Pitch Synchrony — Participant {participant_id}", fontsize=16)

    for idx, (body_key, label) in enumerate(body_parts.items()):
        df1, df2 = df_dict.get(body_key, (None, None))
        ax = axes[idx]

        if df1 is None or df2 is None or df1.empty or df2.empty:
            ax.set_title(f"{label}: ❌ No Data")
            ax.axis("off")
            continue

        merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
        if merged.empty:
            ax.set_title(f"{label}: ❌ No Overlapping Time")
            ax.axis("off")
            continue

        pitch_chris = normalize(merged["rotation_pitch_chris"])
        pitch_pawn = normalize(merged["rotation_pitch_pawn"])
        time = merged["time"]

        pitch_diff = np.abs(pitch_chris - pitch_pawn)
        in_sync = pitch_diff < 0.15
        sync_ratio = in_sync.sum() / len(in_sync) * 100

        ax.plot(time, pitch_chris, label=f"{char1_name} Pitch", color="blue")
        ax.plot(time, pitch_pawn, label=f"{char2_name} Pitch", color="orange")
        ax.fill_between(time, pitch_chris, pitch_pawn, where=in_sync,
                        color="green", alpha=0.3,
                        label=f"In Sync — {sync_ratio:.1f}%")

        ax.set_title(f"{label} Synchrony — {sync_ratio:.1f}%", fontsize=12)
        ax.set_ylabel("Normalized Pitch")
        ax.grid(True)
        ax.legend(loc="upper right")

    plt.xlabel(f"⏱ Time (aligned) — Participant {participant_id}")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    fname = f"{participant_id}_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, fname), dpi=300)
    plt.close()

# === MAIN SCRIPT ===

participant_data = defaultdict(list)

print("🔍 Scanning folder structure...")

for participant_id in os.listdir(ROOT):
    participant_path = os.path.join(ROOT, participant_id)
    if not os.path.isdir(participant_path):
        continue

    heat_logs_path = os.path.join(participant_path, "heat_logs")
    if not os.path.exists(heat_logs_path):
        continue

    for session_name in os.listdir(heat_logs_path):
        session_path = os.path.join(heat_logs_path, session_name)
        if not os.path.isdir(session_path):
            continue

        csv_files = [f for f in os.listdir(session_path) if f.endswith(".csv")]
        if not csv_files:
            print(f"⚠️ No CSV files in {participant_id}/{session_name}")
            continue

        session_dfs = []
        for file in csv_files:
            file_path = os.path.join(session_path, file)
            try:
                df = pd.read_csv(file_path)
                session_dfs.append(df)
            except Exception as e:
                print(f"❌ Failed to read {file_path}: {e}")

        if session_dfs:
            session_df = pd.concat(session_dfs, ignore_index=True)
            participant_data[participant_id].append(session_df)

print(f"\n✅ Found {len(participant_data)} participants with CSV data.\n")

for participant_id, dfs in participant_data.items():
    print(f"📊 Processing Participant {participant_id} ...")

    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_pitch", "Aname"}.issubset(full_df.columns):
        print(f"⚠️ Skipping {participant_id}: missing required columns.")
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()

    if len(characters) < 2:
        print(f"⚠️ Skipping {participant_id}: less than two characters found.")
        continue

    char1, char2 = characters[:2]

    df_dict = {}
    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
        df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

        df_dict[body_part] = (df1, df2)

    plot_combined_sync(df_dict, participant_id, char1, char2)

print("\n✅ All plots saved to:", OUTPUT_DIR)


🔍 Scanning folder structure...
⚠️ No CSV files in 2150/2025-6-5_101933
⚠️ No CSV files in 2150/2025-6-5_102122
⚠️ No CSV files in 2150/2025-6-5_13161
⚠️ No CSV files in 3709/2025-6-4_83728
⚠️ No CSV files in 4050/2025-6-5_152940
⚠️ No CSV files in 4050/2025-6-5_153025
⚠️ No CSV files in 5390/2025-6-5_124511
⚠️ No CSV files in 5390/2025-6-5_124612
⚠️ No CSV files in 5390/2025-6-5_124740
⚠️ No CSV files in 5390/2025-6-5_12527
⚠️ No CSV files in 5985/2025-6-6_142335
⚠️ No CSV files in 9084/2025-6-7_13249
⚠️ No CSV files in 9383/2025-6-6_154812

✅ Found 24 participants with CSV data.

📊 Processing Participant 1060 ...


C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129504 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129306 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 9201 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 9995 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 127898 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:201:

📊 Processing Participant 2150 ...


C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129504 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129306 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 9201 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 9995 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 127898 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:201:

📊 Processing Participant 2182 ...


...

📊 Processing Participant 9345 ...


C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129504 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129306 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 9201 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 9995 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 127898 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:201:

📊 Processing Participant 9383 ...


C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129504 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129306 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 9201 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 9995 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 127898 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Program Files (x86)\Anaconda\lib\site-packages\matplotlib\backends\backend_agg.py:201:


✅ All plots saved to: C:\Users\bmoha\Desktop\DataCollection\sync_plots


In [13]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Adjust to your actual path
OUTPUT_DIR = os.path.join(ROOT, "sync_plots")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_combined_sync(df_dict, participant_id, char1_name, char2_name):
    body_parts = {
        "Head": "Head",
        "WristLeft": "Wrist Left",
        "WristRight": "Wrist Right"
    }

    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    fig.suptitle(f"Pitch Synchrony — Participant {participant_id}", fontsize=16)

    for idx, (body_key, label) in enumerate(body_parts.items()):
        df1, df2 = df_dict.get(body_key, (None, None))
        ax = axes[idx]

        if df1 is None or df2 is None or df1.empty or df2.empty:
            ax.set_title(f"{label}: No Data")
            ax.axis("off")
            continue

        merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
        if merged.empty:
            ax.set_title(f"{label}: No Overlapping Time")
            ax.axis("off")
            continue

        pitch_chris = normalize(merged["rotation_pitch_chris"])
        pitch_pawn = normalize(merged["rotation_pitch_pawn"])
        time = merged["time"]

        pitch_diff = np.abs(pitch_chris - pitch_pawn)
        in_sync = pitch_diff < 0.15
        sync_ratio = in_sync.sum() / len(in_sync) * 100

        ax.plot(time, pitch_chris, label=f"{char1_name} Pitch", color="blue")
        ax.plot(time, pitch_pawn, label=f"{char2_name} Pitch", color="orange")
        ax.fill_between(time, pitch_chris, pitch_pawn, where=in_sync,
                        color="green", alpha=0.3,
                        label=f"In Sync — {sync_ratio:.1f}%")

        ax.set_title(f"{label} Synchrony — {sync_ratio:.1f}%", fontsize=12)
        ax.set_ylabel("Normalized Pitch")
        ax.grid(True)
        ax.legend(loc="upper right")

    plt.xlabel(f"Time (aligned) — Participant {participant_id}")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    fname = f"{participant_id}_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, fname), dpi=300)
    plt.close()

# === MAIN SCRIPT ===

participant_data = defaultdict(list)

print("🔍 Scanning folder structure...")

for participant_id in os.listdir(ROOT):
    participant_path = os.path.join(ROOT, participant_id)
    if not os.path.isdir(participant_path):
        continue

    heat_logs_path = os.path.join(participant_path, "heat_logs")
    if not os.path.exists(heat_logs_path):
        continue

    for session_name in os.listdir(heat_logs_path):
        session_path = os.path.join(heat_logs_path, session_name)
        if not os.path.isdir(session_path):
            continue

        csv_files = [f for f in os.listdir(session_path) if f.endswith(".csv")]
        if not csv_files:
            print(f"⚠️ No CSV files in {participant_id}/{session_name}")
            continue

        session_dfs = []
        for file in csv_files:
            file_path = os.path.join(session_path, file)
            try:
                df = pd.read_csv(file_path)
                session_dfs.append(df)
            except Exception as e:
                print(f"❌ Failed to read {file_path}: {e}")

        if session_dfs:
            session_df = pd.concat(session_dfs, ignore_index=True)
            participant_data[participant_id].append(session_df)

print(f"\n✅ Found {len(participant_data)} participants with CSV data.\n")

for participant_id, dfs in participant_data.items():
    print(f"📊 Processing Participant {participant_id} ...")

    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_pitch", "Aname"}.issubset(full_df.columns):
        print(f"⚠️ Skipping {participant_id}: missing required columns.")
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()

    if len(characters) < 2:
        print(f"⚠️ Skipping {participant_id}: less than two characters found.")
        continue

    char1, char2 = characters[:2]

    df_dict = {}
    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
        df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

        df_dict[body_part] = (df1, df2)

    plot_combined_sync(df_dict, participant_id, char1, char2)

print("\n✅ All plots saved to:", OUTPUT_DIR)


🔍 Scanning folder structure...
⚠️ No CSV files in 2150/2025-6-5_101933
⚠️ No CSV files in 2150/2025-6-5_102122
⚠️ No CSV files in 2150/2025-6-5_13161
⚠️ No CSV files in 3709/2025-6-4_83728
⚠️ No CSV files in 4050/2025-6-5_152940
⚠️ No CSV files in 4050/2025-6-5_153025
⚠️ No CSV files in 5390/2025-6-5_124511
⚠️ No CSV files in 5390/2025-6-5_124612
⚠️ No CSV files in 5390/2025-6-5_124740
⚠️ No CSV files in 5390/2025-6-5_12527
⚠️ No CSV files in 5985/2025-6-6_142335
⚠️ No CSV files in 9084/2025-6-7_13249
⚠️ No CSV files in 9383/2025-6-6_154812

✅ Found 24 participants with CSV data.

📊 Processing Participant 1060 ...
📊 Processing Participant 2150 ...
📊 Processing Participant 2182 ...
📊 Processing Participant 2226 ...
📊 Processing Participant 2462 ...
📊 Processing Participant 2973 ...
⚠️ Skipping 2973: less than two characters found.
📊 Processing Participant 3154 ...
⚠️ Skipping 3154: less than two characters found.
📊 Processing Participant 3291 ...
⚠️ Skipping 3291: less than two characte

In [14]:
import seaborn as sns

# === COLLECT SYNCHRONY DATA FOR SUMMARY CHART ===
sync_summary = []

for participant_id, dfs in participant_data.items():
    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_pitch", "Aname"}.issubset(full_df.columns):
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()

    if len(characters) < 2:
        continue

    char1, char2 = characters[:2]

    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_pitch"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_pitch": "rotation_pitch_chris"})
        df2 = df2.rename(columns={"rotation_pitch": "rotation_pitch_pawn"})

        merged = pd.merge(df1, df2, on="time")
        if merged.empty:
            continue

        pitch_chris = normalize(merged["rotation_pitch_chris"])
        pitch_pawn = normalize(merged["rotation_pitch_pawn"])
        pitch_diff = np.abs(pitch_chris - pitch_pawn)
        in_sync = pitch_diff < 0.15
        sync_ratio = in_sync.sum() / len(in_sync) * 100

        # Append to summary
        sync_summary.append({
            "Participant": participant_id,
            "BodyPart": body_part,
            "SyncPercent": round(sync_ratio, 2)
        })

# === CONVERT TO DATAFRAME ===
summary_df = pd.DataFrame(sync_summary)

# === SAVE SUMMARY CSV ===
csv_path = os.path.join(OUTPUT_DIR, "synchrony_summary.csv")
summary_df.to_csv(csv_path, index=False)
print(f"📄 Synchrony summary saved to {csv_path}")

# === PLOT BAR CHART ===
plt.figure(figsize=(14, 6))
sns.barplot(data=summary_df, x="Participant", y="SyncPercent", hue="BodyPart")
plt.title("Pitch Synchrony % by Participant and Body Part")
plt.ylabel("% Time in Sync (ΔPitch < 0.15)")
plt.xlabel("Participant")
plt.xticks(rotation=45)
plt.legend(title="Body Part")
plt.tight_layout()

# Save chart
chart_path = os.path.join(OUTPUT_DIR, "synchrony_comparison_chart.png")
plt.savefig(chart_path, dpi=300)
plt.close()

print(f"📊 Comparison chart saved to {chart_path}")


📄 Synchrony summary saved to C:\Users\bmoha\Desktop\DataCollection\sync_plots\synchrony_summary.csv
📊 Comparison chart saved to C:\Users\bmoha\Desktop\DataCollection\sync_plots\synchrony_comparison_chart.png


yaw plots


# Yaw Plots

In [15]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Adjust to your path
OUTPUT_DIR = os.path.join(ROOT, "sync_plots_yaw")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILITIES ===

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_combined_sync_yaw(df_dict, participant_id, char1_name, char2_name):
    body_parts = {
        "Head": "Head",
        "WristLeft": "Wrist Left",
        "WristRight": "Wrist Right"
    }

    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    fig.suptitle(f"Yaw Synchrony — Participant {participant_id}", fontsize=16)

    for idx, (body_key, label) in enumerate(body_parts.items()):
        df1, df2 = df_dict.get(body_key, (None, None))
        ax = axes[idx]

        if df1 is None or df2 is None or df1.empty or df2.empty:
            ax.set_title(f"{label}: No Data")
            ax.axis("off")
            continue

        merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
        if merged.empty:
            ax.set_title(f"{label}: No Overlapping Time")
            ax.axis("off")
            continue

        yaw_chris = normalize(merged["rotation_yaw_chris"])
        yaw_pawn = normalize(merged["rotation_yaw_pawn"])
        time = merged["time"]

        yaw_diff = np.abs(yaw_chris - yaw_pawn)
        in_sync = yaw_diff < 0.15
        sync_ratio = in_sync.sum() / len(in_sync) * 100

        ax.plot(time, yaw_chris, label=f"{char1_name} Yaw", color="blue")
        ax.plot(time, yaw_pawn, label=f"{char2_name} Yaw", color="orange")
        ax.fill_between(time, yaw_chris, yaw_pawn, where=in_sync,
                        color="green", alpha=0.3,
                        label=f"In Sync — {sync_ratio:.1f}%")

        ax.set_title(f"{label} Yaw Synchrony — {sync_ratio:.1f}%", fontsize=12)
        ax.set_ylabel("Normalized Yaw")
        ax.grid(True)
        ax.legend(loc="upper right")

    plt.xlabel(f"Time (aligned) — Participant {participant_id}")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    fname = f"{participant_id}_yaw_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, fname), dpi=300)
    plt.close()

# === MAIN SCRIPT ===

participant_data = defaultdict(list)

print("🔍 Scanning folder structure...")

for participant_id in os.listdir(ROOT):
    participant_path = os.path.join(ROOT, participant_id)
    if not os.path.isdir(participant_path):
        continue

    heat_logs_path = os.path.join(participant_path, "heat_logs")
    if not os.path.exists(heat_logs_path):
        continue

    for session_name in os.listdir(heat_logs_path):
        session_path = os.path.join(heat_logs_path, session_name)
        if not os.path.isdir(session_path):
            continue

        csv_files = [f for f in os.listdir(session_path) if f.endswith(".csv")]
        if not csv_files:
            print(f"⚠️ No CSV files in {participant_id}/{session_name}")
            continue

        session_dfs = []
        for file in csv_files:
            file_path = os.path.join(session_path, file)
            try:
                df = pd.read_csv(file_path)
                session_dfs.append(df)
            except Exception as e:
                print(f"❌ Failed to read {file_path}: {e}")

        if session_dfs:
            session_df = pd.concat(session_dfs, ignore_index=True)
            participant_data[participant_id].append(session_df)

print(f"\n✅ Found {len(participant_data)} participants with CSV data.\n")

for participant_id, dfs in participant_data.items():
    print(f"📊 Processing Participant {participant_id} ...")

    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_yaw", "Aname"}.issubset(full_df.columns):
        print(f"⚠️ Skipping {participant_id}: missing required yaw columns.")
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()

    if len(characters) < 2:
        print(f"⚠️ Skipping {participant_id}: less than two characters found.")
        continue

    char1, char2 = characters[:2]

    df_dict = {}
    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_yaw"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_yaw"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_yaw": "rotation_yaw_chris"})
        df2 = df2.rename(columns={"rotation_yaw": "rotation_yaw_pawn"})

        df_dict[body_part] = (df1, df2)

    plot_combined_sync_yaw(df_dict, participant_id, char1, char2)

print("\n✅ All yaw plots saved to:", OUTPUT_DIR)


🔍 Scanning folder structure...
⚠️ No CSV files in 2150/2025-6-5_101933
⚠️ No CSV files in 2150/2025-6-5_102122
⚠️ No CSV files in 2150/2025-6-5_13161
⚠️ No CSV files in 3709/2025-6-4_83728
⚠️ No CSV files in 4050/2025-6-5_152940
⚠️ No CSV files in 4050/2025-6-5_153025
⚠️ No CSV files in 5390/2025-6-5_124511
⚠️ No CSV files in 5390/2025-6-5_124612
⚠️ No CSV files in 5390/2025-6-5_124740
⚠️ No CSV files in 5390/2025-6-5_12527
⚠️ No CSV files in 5985/2025-6-6_142335
⚠️ No CSV files in 9084/2025-6-7_13249
⚠️ No CSV files in 9383/2025-6-6_154812

✅ Found 24 participants with CSV data.

📊 Processing Participant 1060 ...
📊 Processing Participant 2150 ...
📊 Processing Participant 2182 ...
📊 Processing Participant 2226 ...
📊 Processing Participant 2462 ...
📊 Processing Participant 2973 ...
⚠️ Skipping 2973: less than two characters found.
📊 Processing Participant 3154 ...
⚠️ Skipping 3154: less than two characters found.
📊 Processing Participant 3291 ...
⚠️ Skipping 3291: less than two characte

In [16]:
import seaborn as sns

# === COLLECT YAW SYNCHRONY FOR CSV + CHART ===
yaw_sync_summary = []

for participant_id, dfs in participant_data.items():
    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_yaw", "Aname"}.issubset(full_df.columns):
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()
    if len(characters) < 2:
        continue
    char1, char2 = characters[:2]

    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_yaw"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_yaw"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_yaw": "rotation_yaw_chris"})
        df2 = df2.rename(columns={"rotation_yaw": "rotation_yaw_pawn"})

        merged = pd.merge(df1, df2, on="time")
        if merged.empty:
            continue

        yaw_chris = normalize(merged["rotation_yaw_chris"])
        yaw_pawn = normalize(merged["rotation_yaw_pawn"])
        yaw_diff = np.abs(yaw_chris - yaw_pawn)
        in_sync = yaw_diff < 0.15
        sync_ratio = in_sync.sum() / len(in_sync) * 100

        yaw_sync_summary.append({
            "Participant": participant_id,
            "BodyPart": body_part,
            "YawSyncPercent": round(sync_ratio, 2)
        })

# === SAVE CSV ===
yaw_df = pd.DataFrame(yaw_sync_summary)
yaw_csv_path = os.path.join(OUTPUT_DIR, "yaw_synchrony_summary.csv")
yaw_df.to_csv(yaw_csv_path, index=False)
print(f"\n📄 Yaw synchrony summary saved to {yaw_csv_path}")

# === OPTIONAL: VISUALIZE YAW SYNCHRONY ACROSS PARTICIPANTS ===
plt.figure(figsize=(14, 6))
sns.barplot(data=yaw_df, x="Participant", y="YawSyncPercent", hue="BodyPart")
plt.title("Yaw Synchrony % by Participant and Body Part")
plt.ylabel("% Time in Sync (ΔYaw < 0.15)")
plt.xlabel("Participant")
plt.xticks(rotation=45)
plt.legend(title="Body Part")
plt.tight_layout()

# Save chart
chart_path = os.path.join(OUTPUT_DIR, "yaw_synchrony_comparison_chart.png")
plt.savefig(chart_path, dpi=300)
plt.close()
print(f"📊 Yaw synchrony bar chart saved to {chart_path}")



📄 Yaw synchrony summary saved to C:\Users\bmoha\Desktop\DataCollection\sync_plots_yaw\yaw_synchrony_summary.csv
📊 Yaw synchrony bar chart saved to C:\Users\bmoha\Desktop\DataCollection\sync_plots_yaw\yaw_synchrony_comparison_chart.png


# Roll plot

In [17]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Adjust if needed
OUTPUT_DIR = os.path.join(ROOT, "sync_plots_roll")
os.makedirs(OUTPUT_DIR, exist_ok=True)

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def extract_character_id(aname):
    return "_".join(aname.split("_")[:3])  # e.g., 'myMRPawn_C_2147482403'

def plot_combined_sync_roll(df_dict, participant_id, char1_name, char2_name):
    body_parts = {
        "Head": "Head",
        "WristLeft": "Wrist Left",
        "WristRight": "Wrist Right"
    }

    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    fig.suptitle(f"Roll Synchrony — Participant {participant_id}", fontsize=16)

    for idx, (body_key, label) in enumerate(body_parts.items()):
        df1, df2 = df_dict.get(body_key, (None, None))
        ax = axes[idx]

        if df1 is None or df2 is None or df1.empty or df2.empty:
            ax.set_title(f"{label}: No Data")
            ax.axis("off")
            continue

        merged = pd.merge(df1, df2, on="time", suffixes=("_chris", "_pawn"))
        if merged.empty:
            ax.set_title(f"{label}: No Overlapping Time")
            ax.axis("off")
            continue

        roll_chris = normalize(merged["rotation_roll_chris"])
        roll_pawn = normalize(merged["rotation_roll_pawn"])
        time = merged["time"]

        roll_diff = np.abs(roll_chris - roll_pawn)
        in_sync = roll_diff < 0.15
        sync_ratio = in_sync.sum() / len(in_sync) * 100

        ax.plot(time, roll_chris, label=f"{char1_name} Roll", color="blue")
        ax.plot(time, roll_pawn, label=f"{char2_name} Roll", color="orange")
        ax.fill_between(time, roll_chris, roll_pawn, where=in_sync,
                        color="green", alpha=0.3,
                        label=f"In Sync — {sync_ratio:.1f}%")

        ax.set_title(f"{label} Roll Synchrony — {sync_ratio:.1f}%", fontsize=12)
        ax.set_ylabel("Normalized Roll")
        ax.grid(True)
        ax.legend(loc="upper right")

    plt.xlabel(f"Time (aligned) — Participant {participant_id}")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    fname = f"{participant_id}_roll_sync.png"
    plt.savefig(os.path.join(OUTPUT_DIR, fname), dpi=300)
    plt.close()

# === PROCESSING PARTICIPANTS ===

participant_data = defaultdict(list)

print("🔍 Scanning folder structure...")

for participant_id in os.listdir(ROOT):
    participant_path = os.path.join(ROOT, participant_id)
    if not os.path.isdir(participant_path):
        continue

    heat_logs_path = os.path.join(participant_path, "heat_logs")
    if not os.path.exists(heat_logs_path):
        continue

    for session_name in os.listdir(heat_logs_path):
        session_path = os.path.join(heat_logs_path, session_name)
        if not os.path.isdir(session_path):
            continue

        csv_files = [f for f in os.listdir(session_path) if f.endswith(".csv")]
        if not csv_files:
            print(f"⚠️ No CSV files in {participant_id}/{session_name}")
            continue

        session_dfs = []
        for file in csv_files:
            file_path = os.path.join(session_path, file)
            try:
                df = pd.read_csv(file_path)
                session_dfs.append(df)
            except Exception as e:
                print(f"❌ Failed to read {file_path}: {e}")

        if session_dfs:
            session_df = pd.concat(session_dfs, ignore_index=True)
            participant_data[participant_id].append(session_df)

print(f"\n✅ Found {len(participant_data)} participants with CSV data.\n")

# === PLOT & SUMMARY ===

roll_sync_summary = []

for participant_id, dfs in participant_data.items():
    print(f"📊 Processing Participant {participant_id} ...")
    full_df = pd.concat(dfs, ignore_index=True)
    if not {"time", "rotation_roll", "Aname"}.issubset(full_df.columns):
        print(f"⚠️ Skipping {participant_id}: missing required columns.")
        continue

    full_df["char_id"] = full_df["Aname"].apply(extract_character_id)
    full_df["body_part"] = full_df["Aname"].apply(lambda x: x.split("_")[-1])
    characters = full_df["char_id"].unique()
    if len(characters) < 2:
        print(f"⚠️ Skipping {participant_id}: less than two characters found.")
        continue

    char1, char2 = characters[:2]

    df_dict = {}
    for body_part in ["Head", "WristLeft", "WristRight"]:
        df1 = full_df[(full_df["char_id"] == char1) & (full_df["body_part"] == body_part)]
        df2 = full_df[(full_df["char_id"] == char2) & (full_df["body_part"] == body_part)]

        df1 = df1[["time", "rotation_roll"]].drop_duplicates("time").sort_values("time")
        df2 = df2[["time", "rotation_roll"]].drop_duplicates("time").sort_values("time")

        df1 = df1.rename(columns={"rotation_roll": "rotation_roll_chris"})
        df2 = df2.rename(columns={"rotation_roll": "rotation_roll_pawn"})

        df_dict[body_part] = (df1, df2)

        # Calculate synchrony %
        merged = pd.merge(df1, df2, on="time")
        if not merged.empty:
            roll_chris = normalize(merged["rotation_roll_chris"])
            roll_pawn = normalize(merged["rotation_roll_pawn"])
            roll_diff = np.abs(roll_chris - roll_pawn)
            in_sync = roll_diff < 0.15
            sync_ratio = in_sync.sum() / len(in_sync) * 100
            roll_sync_summary.append({
                "Participant": participant_id,
                "BodyPart": body_part,
                "RollSyncPercent": round(sync_ratio, 2)
            })

    plot_combined_sync_roll(df_dict, participant_id, char1, char2)

# === SAVE CSV ===
summary_df = pd.DataFrame(roll_sync_summary)
csv_path = os.path.join(OUTPUT_DIR, "roll_synchrony_summary.csv")
summary_df.to_csv(csv_path, index=False)
print(f"\n📄 Roll synchrony summary saved to {csv_path}")

# === BAR CHART COMPARISON ===
plt.figure(figsize=(14, 6))
sns.barplot(data=summary_df, x="Participant", y="RollSyncPercent", hue="BodyPart")
plt.title("Roll Synchrony % by Participant and Body Part")
plt.ylabel("% Time in Sync (ΔRoll < 0.15)")
plt.xlabel("Participant")
plt.xticks(rotation=45)
plt.legend(title="Body Part")
plt.tight_layout()

chart_path = os.path.join(OUTPUT_DIR, "roll_synchrony_comparison_chart.png")
plt.savefig(chart_path, dpi=300)
plt.close()
print(f"📊 Roll synchrony bar chart saved to {chart_path}")


🔍 Scanning folder structure...
⚠️ No CSV files in 2150/2025-6-5_101933
⚠️ No CSV files in 2150/2025-6-5_102122
⚠️ No CSV files in 2150/2025-6-5_13161
⚠️ No CSV files in 3709/2025-6-4_83728
⚠️ No CSV files in 4050/2025-6-5_152940
⚠️ No CSV files in 4050/2025-6-5_153025
⚠️ No CSV files in 5390/2025-6-5_124511
⚠️ No CSV files in 5390/2025-6-5_124612
⚠️ No CSV files in 5390/2025-6-5_124740
⚠️ No CSV files in 5390/2025-6-5_12527
⚠️ No CSV files in 5985/2025-6-6_142335
⚠️ No CSV files in 9084/2025-6-7_13249
⚠️ No CSV files in 9383/2025-6-6_154812

✅ Found 24 participants with CSV data.

📊 Processing Participant 1060 ...
📊 Processing Participant 2150 ...
📊 Processing Participant 2182 ...
📊 Processing Participant 2226 ...
📊 Processing Participant 2462 ...
📊 Processing Participant 2973 ...
⚠️ Skipping 2973: less than two characters found.
📊 Processing Participant 3154 ...
⚠️ Skipping 3154: less than two characters found.
📊 Processing Participant 3291 ...
⚠️ Skipping 3291: less than two characte

In [18]:
import pandas as pd
import os

# === Paths to your summary CSVs ===
root_dir = r"C:\Users\bmoha\Desktop\DataCollection"
pitch_path = os.path.join(root_dir, "sync_plots", "synchrony_summary.csv")
yaw_path = os.path.join(root_dir, "sync_plots_yaw", "yaw_synchrony_summary.csv")
roll_path = os.path.join(root_dir, "sync_plots_roll", "roll_synchrony_summary.csv")

# === Load each summary ===
pitch_df = pd.read_csv(pitch_path).rename(columns={"SyncPercent": "PitchSyncPercent"})
yaw_df   = pd.read_csv(yaw_path).rename(columns={"YawSyncPercent": "YawSyncPercent"})
roll_df  = pd.read_csv(roll_path).rename(columns={"RollSyncPercent": "RollSyncPercent"})

# === Merge on Participant and BodyPart ===
merged = pitch_df.merge(yaw_df, on=["Participant", "BodyPart"], how="outer")
merged = merged.merge(roll_df, on=["Participant", "BodyPart"], how="outer")

# === Save the combined summary ===
combined_path = os.path.join(root_dir, "combined_synchrony_summary.csv")
merged.to_csv(combined_path, index=False)
print(f"✅ Combined synchrony summary saved to:\n{combined_path}")


✅ Combined synchrony summary saved to:
C:\Users\bmoha\Desktop\DataCollection\combined_synchrony_summary.csv
